In [5]:
!pip install libs_qrem
!pip install qiskit-ignis   

In [12]:
from qiskit import IBMQ
IBMQ.save_account('872395c64aae629a782c0f29e240279b70d8831a8a57be2271e6bc43a62dc9760e9029602bd94b411651319af154544a5fa015225362d0574d77d193720673b2', overwrite=True)
IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-11-29 00:00:03,852: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [15]:
from qiskit import IBMQ, Aer
provider = IBMQ.load_account()
available_cloud_backends = provider.backends() 
print('\nHere is the list of cloud backends that are available to you:')
for i in available_cloud_backends: print(i)

available_local_backends = Aer.backends() 
print('\nHere is the list of local backends that are available to you: ')
for i in available_local_backends: print(i)

ibmqfactory.load_account:WARNING:2022-11-29 00:03:55,148: Credentials are already in use. The existing account in the session will be replaced.



Here is the list of cloud backends that are available to you:
ibmq_qasm_simulator
ibmq_lima
ibmq_belem
ibmq_quito
simulator_statevector
simulator_mps
simulator_extended_stabilizer
simulator_stabilizer
ibmq_manila
ibm_nairobi
ibm_oslo

Here is the list of local backends that are available to you: 
aer_simulator
aer_simulator_statevector
aer_simulator_density_matrix
aer_simulator_stabilizer
aer_simulator_matrix_product_state
aer_simulator_extended_stabilizer
aer_simulator_unitary
aer_simulator_superop
qasm_simulator
statevector_simulator
unitary_simulator
pulse_simulator


In [25]:
from qiskit import QuantumRegister, Aer, execute
from qiskit.ignis.mitigation.measurement import tensored_meas_cal
# prepare calibration circuit (same as qiskit tutorial)
n = 5
qr = QuantumRegister(n)
mit_pattern = [[2], [3, 4]]
meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')

# run calibration circuit (same as qiskit tutorial)
backend = Aer.get_backend('aer_simulator')
job = execute(meas_calibs, backend=backend, shots=5000)
cal_results = job.result()

from qiskit.ignis.mitigation.measurement import TensoredMeasFitter
meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)
print(meas_fitter.cal_matrices )
# Create mitigator instance (corresponds to meas_fitter.filter in the tutorial code.)
# this is very similar to the usage of qiskit.ignis.mitigation modules 
# meas_filter = meas_fitter.filter
from libs_qrem import LeastNormFilter
meas_filter = LeastNormFilter(n, meas_fitter.cal_matrices)

# apply mitigation
# Let `noisy_hist` be a dict variable representing a noisy histogram obtained from `.get_counts()` method in `qiskit.result.Result` instance.
noisy_hist = {"000": 50, "101": 20, "111": 30}
# Then you can mitigate it as follows.
mitigated_hist = meas_filter.apply(noisy_hist)


[array([[1., 0.],
       [0., 1.]]), array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])]


In [26]:
print(mitigated_hist)

{'000': 33.333333333333336, '101': 33.333333333333336, '111': 33.333333333333336}
